### Hanqing Zhao
#### CSE 258 HW3

In [304]:
import gzip
from collections import defaultdict
import math
#import tensorflow as tf
import scipy.optimize
from sklearn import svm
import numpy as np
from tqdm import tqdm
import string
import random
import string
from sklearn import linear_model

In [296]:
import warnings
warnings.filterwarnings("ignore")

In [297]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [298]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [299]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [163]:
answers = {}

In [164]:
# Some data structures that will be useful

In [300]:
allRatings = []
for l in readCSV("assignment1/train_Interactions.csv.gz"):
    allRatings.append(l)

In [301]:
len(allRatings)

200000

In [302]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [69]:
allRatings[0]

('u67805239', 'b61372131', 4)

In [70]:
##################################################
# Rating prediction (CSE258 only)                #
##################################################

In [71]:
### Question 9

In [329]:
N = len(ratingsTrain)
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())
alpha = sum([ratingsTrain[i][2] for i in range(len(ratingsTrain))])/len(ratingsTrain)
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [333]:
def predict(user, item):
    p = alpha + userBiases[user] + itemBiases[item]
    return p

def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))
    
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [predict(d[0], d[1]) for d in ratingsTrain]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost
    
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(ratingsTrain)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in ratingsTrain:
        u,i = d[0], d[1]
        pred = predict(u, i)
        diff = pred - d[2]
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)


In [334]:
train_labels = [d[2] for d in ratingsTrain]
#use lambda given as 1
lamb = 1

In [335]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (train_labels, lamb))

MSE = 1.741577477865528
MSE = 1.719164669097517
MSE = 1.7412760775882776
MSE = 1.7412760171399868


(array([ 3.68666543e+00, -2.49296541e-04,  9.89577346e-06, ...,
         6.91219566e-06,  1.64853603e-06,  1.64859815e-06]),
 1.7414265936521982,
 {'grad': array([ 4.45195907e-07, -9.02043271e-08, -6.31721614e-09, ...,
          2.91971874e-10, -1.05126621e-09, -1.05166986e-09]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 4,
  'nit': 2,
  'warnflag': 0})

In [339]:
#calculate the MSE for the validation dataset ratingsValid
pred = []
for d in ratingsValid:
    u, i = d[0], d[1]
    if u in userBiases and i in itemBiases:
        pred.append(predict(u, i))
    else:
        pred.append(0)

valid_labels = [d[2] for d in ratingsValid]
validMSE = MSE(pred, valid_labels)
print("MSE %.3f" % MSE(pred, valid_labels))

MSE 1.719


In [340]:
answers['Q9'] = validMSE

In [341]:
assertFloat(answers['Q9'])

In [342]:
### Question 10

In [363]:
userBiases

{'u67805239': -0.00024929654107228396,
 'u54531895': 9.89577346091438e-06,
 'u76549666': -1.912946802526045e-06,
 'u03186275': -4.6676820156865394e-05,
 'u21322233': 1.4839733898312239e-05,
 'u00402241': 2.976314647069468e-06,
 'u88999268': 1.3290272025539897e-06,
 'u39455611': 8.097340891765092e-05,
 'u90502882': 9.887924047872421e-06,
 'u92679832': 2.7004565296273144e-05,
 'u88271768': -2.619719896589834e-06,
 'u42487937': 2.1727619593434983e-05,
 'u50260163': 1.8769540196742687e-05,
 'u76802936': 6.276214608288586e-06,
 'u53795404': -1.966131358127734e-06,
 'u63958609': 4.083027395652446e-05,
 'u22899611': -1.674597407988506e-05,
 'u50844125': 0.00018660105008827215,
 'u11614716': 4.635841138654903e-06,
 'u09810497': -1.3134525294574001e-05,
 'u73528797': 1.329228986529743e-05,
 'u71509810': 4.512807072820032e-05,
 'u81603581': 3.981337657855445e-06,
 'u28722937': -1.4409771247807998e-05,
 'u62762345': 1.4831783402269346e-05,
 'u45384323': 5.637309043725535e-06,
 'u10740755': -1.643

In [356]:
maxUser = str(max(userBiases, key=userBiases.get))
minUser = str(min(userBiases, key=userBiases.get))
maxBeta = float(max(userBiases.values()))
minBeta = float(min(userBiases.values()))

In [357]:
answers['Q10'] = [maxUser, minUser, maxBeta, minBeta]

In [358]:
answers['Q10']

['u71190649', 'u56426501', 0.00021919259774434816, -0.00046570331342318566]

In [359]:
assert [type(x) for x in answers['Q10']] == [str, str, float, float]

In [360]:
### Question 11

In [365]:
#I chose a smaller lambda for better MSE
lamb_ = 0.001
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (train_labels, lamb_))

MSE = 1.7660556962845448
MSE = 2.4465259431094015
MSE = 1.741230963619969
MSE = 1.740929114441745
MSE = 1.7397279142240711
MSE = 1.7350222572823735
MSE = 1.7177859323796927
MSE = 1.6497758636566833
MSE = 1.632990693986021
MSE = 1.6236090417939872
MSE = 1.6261039891388702
MSE = 1.6263390844800325
MSE = 1.6262169562352446
MSE = 1.6261515240037743
MSE = 1.6261441159523389


(array([ 3.63132731e+00, -2.28864005e-01,  1.16027062e-02, ...,
         6.97963104e-03,  1.88481815e-03,  1.92154737e-03]),
 1.6710666928550408,
 {'grad': array([-5.89093127e-08, -2.69887607e-07, -2.46801804e-08, ...,
         -1.91344727e-08, -7.06503858e-09, -8.49263452e-09]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 15,
  'nit': 10,
  'warnflag': 0})

In [373]:
#calculate the MSE again for the valid dataset
pred_ = []
for d in ratingsValid:
    u, i = d[0], d[1]
    if u in userBiases and i in itemBiases:
        pred_.append(predict(u, i))
    else:
        pred_.append(0)

valid_labels = [d[2] for d in ratingsValid]
validMSE = MSE(pred_, valid_labels)
print("MSE %f" % MSE(pred_, valid_labels))

MSE 1.632507


In [368]:
answers['Q11'] = (lamb_, validMSE)

In [369]:
assertFloat(answers['Q11'][0])
assertFloat(answers['Q11'][1])

In [361]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("assignment1/pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    
predictions.close()

In [86]:
##################################################
# Read prediction                                #
##################################################

In [374]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("assignment1/train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [375]:
### Question 1

In [376]:
#negSample = ratingsValid
negSamples = []
totalBooks = [row[1] for row in allRatings]

In [377]:
len(ratingsValid)

10000

In [378]:
for i in tqdm(range(len(ratingsValid))):
    user = ratingsValid[i][0]
    userBooks = [t[0] for t in ratingsPerUser[user]]
    neg_bookID = random.sample(set(totalBooks) - set(userBooks), 1)[0]
    #unread book ratings as 0?
    negSamples.append((user, neg_bookID, 0))


100%|██████████| 10000/10000 [01:29<00:00, 112.21it/s]


In [379]:
len(negSamples)

10000

In [380]:
valid_label = [1 for i in range(len(ratingsValid))]
len(valid_label)

10000

In [381]:
neg_label = [0 for i in range(len(negSamples))]
len(neg_label)

10000

In [382]:
total_label = valid_label+neg_label

In [383]:
ratingsValid_ = ratingsValid + negSamples

In [384]:
len(ratingsValid_)

20000

In [385]:
predictions = []    
for t in ratingsValid_:
    if t[1] in return1:
        predictions.append(1)
    else:
        predictions.append(0)

correctPred = np.array(predictions) == np.array(total_label)
acc1 = sum(correctPred) / len(correctPred)
print('Accuracy on Validation set is %f' % acc1)

Accuracy on Validation set is 0.709350


In [386]:
answers['Q1'] = acc1

In [387]:
assertFloat(answers['Q1'])

In [388]:
### Question 2

In [389]:
return1 = set()
count = 0
threshold = 0.1
acc_list = []
#Loop to see which threshold has the best accuracy
for n in range(10):
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalRead * threshold * (n + 1): break
    
    pred = []    
    for t in ratingsValid_:
        if t[1] in return1:
            pred.append(1)
        else:
            pred.append(0)
    
    correctPred = np.array(pred) == np.array(total_label)
    acc = sum(correctPred) / len(correctPred)
    acc_list.append([round(threshold * (n + 1), 2), acc])
    
    return1 = set()
    count = 0


In [390]:
acc_list

[[0.1, 0.54465],
 [0.2, 0.5911],
 [0.3, 0.6352],
 [0.4, 0.6715],
 [0.5, 0.70935],
 [0.6, 0.73515],
 [0.7, 0.75015],
 [0.8, 0.74465],
 [0.9, 0.6924],
 [1.0, 0.5]]

In [391]:
max_acc = 0
threshold = 0
for pair in acc_list:
    if pair[1] > max_acc:
        max_acc = pair[1]
        threshold = pair[0]

print(threshold, max_acc)
acc2 = max_acc

0.7 0.75015


In [392]:
answers['Q2'] = [threshold, acc2]

In [393]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [394]:
### Question 3/4

In [395]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer/denom

In [396]:
def mostSimilar(u, b):
    similarities = []
    books = set([t[0] for t in ratingsPerUser[u]])
    for b2 in books:
        if b2 == b:continue
        users_b = set([t[0] for t in ratingsPerItem[b]])
        users_b2 = set([t[0] for t in ratingsPerItem[b2]])
        sim = Jaccard(users_b, users_b2)
        similarities.append(sim)
    similarities.sort(reverse=True)
    return similarities[:10]

In [397]:
mostSimilar(ratingsValid[3][0], ratingsValid[3][1])

[0.01990049751243781,
 0.006872852233676976,
 0.006329113924050633,
 0.0045662100456621,
 0.0021598272138228943,
 0.0,
 0.0,
 0.0]

In [398]:
#Jaccard similarity
acc_list2 = []
threshold = 0.001

for n in range(0, 30):
    sim_list = []
    prediction = []  
    
    for data in ratingsValid_:
        sim_list = mostSimilar(data[0], data[1])
        
        #if bigger than some threshold, we predict it to be 1
        #print(len(sim_list))
        if len(sim_list) != 0 and sim_list[0] >= threshold * (n + 1):
            prediction.append(1)
        else:
            prediction.append(0)
    
    correctPred = np.array(prediction) == np.array(total_label)
    acc = sum(correctPred) / len(correctPred)
    acc_list2.append([threshold * (n + 1), acc])

In [399]:
acc_list2

[[0.001, 0.68375],
 [0.002, 0.6945],
 [0.003, 0.7003],
 [0.004, 0.69775],
 [0.005, 0.6895],
 [0.006, 0.6782],
 [0.007, 0.6572],
 [0.008, 0.63665],
 [0.009000000000000001, 0.61465],
 [0.01, 0.59475],
 [0.011, 0.5739],
 [0.012, 0.5604],
 [0.013000000000000001, 0.5451],
 [0.014, 0.5353],
 [0.015, 0.5285],
 [0.016, 0.522],
 [0.017, 0.51725],
 [0.018000000000000002, 0.51325],
 [0.019, 0.51],
 [0.02, 0.5074],
 [0.021, 0.5053],
 [0.022, 0.5041],
 [0.023, 0.50375],
 [0.024, 0.50335],
 [0.025, 0.5033],
 [0.026000000000000002, 0.5028],
 [0.027, 0.50275],
 [0.028, 0.50275],
 [0.029, 0.5025],
 [0.03, 0.5021]]

In [400]:
max_acc = 0
threshold = 0
for pair in acc_list2:
    if pair[1] > max_acc:
        max_acc = pair[1]
        threshold = pair[0]

print(threshold, max_acc)
acc3 = max_acc

0.003 0.7003


In [401]:
# Q4 also add the popularity
return1 = set()
count = 0
threshold_pop = 0.7
threshold_jac = 0.003

for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead * threshold_pop: break

prediction = []
sim_list = []

for data in ratingsValid_:
    sim_list = mostSimilar(data[0], data[1])

    if len(sim_list) != 0 and sim_list[0] >= threshold * (n + 1):
        prediction.append(1)
    elif data[1] in return1:
        prediction.append(1)
    else:
        prediction.append(0)

correctPred = np.array(prediction) == np.array(total_label)
acc4 = sum(correctPred) / len(correctPred)

In [402]:
acc4

0.7503

In [403]:
answers['Q3'] = acc3

In [404]:
answers['Q4'] = acc4

In [405]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [406]:
sim_list_ = []
predictions = open("predictions_Read.csv", 'w')
for l in open("assignment1/pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    sim_list_ = mostSimilar(u, b)
    if len(sim_list_)!=0 and sim_list_[0] >= threshold_jac:
        predictions.write(u + ',' + b + ",1\n")
    elif b in return1:
        predictions.write(u + ',' + b + ",1\n")
    else:
        predictions.write(u + ',' + b + ",0\n")

predictions.close()

In [407]:
predictions

<_io.TextIOWrapper name='predictions_Read.csv' mode='w' encoding='UTF-8'>

In [408]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [409]:
assert type(answers['Q5']) == str

In [292]:
##################################################
# Category prediction (CSE158 only)              #
##################################################

In [ ]:
### Question 6

In [ ]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)

In [ ]:
data[0]

In [ ]:
answers['Q6'] = counts[:10]

In [ ]:
assert [type(x[0]) for x in answers['Q6']] == [int]*10
assert [type(x[1]) for x in answers['Q6']] == [str]*10

In [ ]:
### Question 7

In [ ]:
Xtrain = X[:9*len(X)//10]
ytrain = y[:9*len(y)//10]
Xvalid = X[9*len(X)//10:]
yvalid = y[9*len(y)//10:]

In [ ]:
answers['Q7'] = acc7

In [ ]:
assertFloat(answers['Q7'])

In [ ]:
### Question 8

In [ ]:
answers['Q8'] = acc8

In [ ]:
assertFloat(answers['Q8'])

In [ ]:
# Run on test set

In [ ]:
predictions = open("predictions_Category.csv", 'w')
pos = 0

for l in open("pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    # (etc.)

predictions.close()

In [410]:
answers

{'Q1': 0.70935,
 'Q2': [0.7, 0.75015],
 'Q3': 0.7003,
 'Q4': 0.7503,
 'Q5': 'I confirm that I have uploaded an assignment submission to gradescope',
 'Q9': 1.719150504903895,
 'Q10': ['u71190649',
  'u56426501',
  0.00021919259774434816,
  -0.00046570331342318566],
 'Q11': (0.001, 1.6325074282564236)}

In [411]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()